In [2]:
from selenium import webdriver
import pandas as pd
import time

In [8]:
# from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
There is no [win32] chromedriver for browser 92.0.4515 in cache
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.107/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Admin\.wdm\drivers\chromedriver\win32\92.0.4515.107]


In [54]:
data_from_excel = pd.read_excel("Chiến dịch Influencers.xlsx", sheet_name="Thống kê số views theo Link vid", index_col=None)

data = data_from_excel

In [55]:
data = data_from_excel

In [56]:
data.head()

,STT,Mã INF,Kênh hợp tác,Link Video
0,1,INFHUYEN641,Facebook,https://www.facebook.com/nguyen.huyenthu.716/v...
1,2,INFHUYEN641,Facebook,https://www.facebook.com/100008065904636/posts...
2,3,INFNHI376,TikTok,https://www.tiktok.com/@nhyyng/video/698771396...
3,4,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69862...
4,5,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69872...


In [57]:
Tiktok_data = data[data['Link Video'].str.startswith('https://www.tiktok.com/')]

In [58]:
# get rid of unwanted new line and spacing
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace('\n', '', regex=True)
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace(' ', '', regex=True)

# get rid of '?...'
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.split("?", expand=True)[0]

# Some links are from tiktoke main page
Tiktok_data = Tiktok_data[Tiktok_data['Link Video'].str.contains("/video/")]

<ipython-input-58-642b7a96ae3e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace('\n', '', regex=True)
<ipython-input-58-642b7a96ae3e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace(' ', '', regex=True)
<ipython-input-58-642b7a96ae3e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [79]:
list1 = Tiktok_data['Link Video'].values[:4]
list2 = Tiktok_data['Link Video'].values[40:80]
list3 = Tiktok_data['Link Video'].values[80:]

In [80]:
list1

array(['https://www.tiktok.com/@nhyyng/video/6987713962817228059',
       'https://www.tiktok.com/@giangdeyy0/video/6986251851104554266',
       'https://www.tiktok.com/@giangdeyy0/video/6987221320555711770',
       'https://www.tiktok.com/@giangdeyy0/video/6987288042763504922'],
      dtype=object)

In [92]:
# Easily bypass notification

from selenium.webdriver.common.by import By

# Bypass notification

# chrome_options = webdriver.ChromeOptions()
# prefs = {"profile.default_content_setting_values.notifications" : 2}
# chrome_options.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome() #chrome_options=chrome_options)

# Access google chrome and open insta login
driver.get("https://www.tiktok.com/login/")

In [100]:
my_list = []
last_url_check = ""

for link in list1:
    
    base_url = link[:-26]
    
    if (base_url == last_url_check):
        
        videos = driver.find_elements(By.XPATH, '//div[@class="jsx-2261688415 _ratio_wrapper"]/a') #.get_attribute("href")
     
        for video in videos:
            video_url = video.get_attribute("href")
                
            if (video_url == link):
                
                view = video.text

                item = {
                            "view": view,
                            "link": link
                }

                my_list.append(item)

    else:
    
        driver.get(base_url)
        
        last_url_check = base_url
        
        SCROLL_PAUSE_TIME = 2

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:

            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
        time.sleep(4) # to load all the data

        videos = driver.find_elements(By.XPATH, '//div[@class="jsx-2261688415 _ratio_wrapper"]/a') #.get_attribute("href")
     
        for video in videos:
            video_url = video.get_attribute("href")
                
            if (video_url == link):
                
                view = video.text

                item = {
                            "view": view,
                            "link": link
                }

                my_list.append(item)


In [102]:
df = pd.DataFrame(my_list)
df

,view,link
0,101K,https://www.tiktok.com/@nhyyng/video/698771396...
1,,https://www.tiktok.com/@giangdeyy0/video/69862...
2,62.7K,https://www.tiktok.com/@giangdeyy0/video/69872...
3,94.4K,https://www.tiktok.com/@giangdeyy0/video/69872...


In [97]:
driver.get("https://www.tiktok.com/@nhyyng")
        

SCROLL_PAUSE_TIME = 2

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:

    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

time.sleep(5)
    
videos = driver.find_elements(By.XPATH, '//div[@class="jsx-2261688415 _ratio_wrapper"]/a') #.get_attribute("href")

for video in videos:
    video_url = video.get_attribute("href")
    view = video.text

    print(video_url, view)

https://www.tiktok.com/@nhyyng/video/6993285022333652250 107.8K
https://www.tiktok.com/@nhyyng/video/6992467169082625306 3939
https://www.tiktok.com/@nhyyng/video/6991797137382051099 121.9K
https://www.tiktok.com/@nhyyng/video/6989973713554787610 1.3M
https://www.tiktok.com/@nhyyng/video/6989581626418187546 86.5K
https://www.tiktok.com/@nhyyng/video/6988835466917334299 669.2K
https://www.tiktok.com/@nhyyng/video/6988469416409255194 1M
https://www.tiktok.com/@nhyyng/video/6987713962817228059 101K
https://www.tiktok.com/@nhyyng/video/6986603853877742874 64.5K
https://www.tiktok.com/@nhyyng/video/6985127985498049819 93.9K
https://www.tiktok.com/@nhyyng/video/6984395772531969307 58.9K
https://www.tiktok.com/@nhyyng/video/6984005896405814555 73.3K
https://www.tiktok.com/@nhyyng/video/6982937443330805018 23.3K
https://www.tiktok.com/@nhyyng/video/6981415021607783706 75.9K
https://www.tiktok.com/@nhyyng/video/6981072024462478618 9172
https://www.tiktok.com/@nhyyng/video/6980675050085174554 17

https://www.tiktok.com/@nhyyng/video/6866274698951527682 534.1K
https://www.tiktok.com/@nhyyng/video/6865551821931400449 41.5K
https://www.tiktok.com/@nhyyng/video/6865204902482726145 32.5K
https://www.tiktok.com/@nhyyng/video/6863391253401308417 446.8K
https://www.tiktok.com/@nhyyng/video/6863040337686891778 20.8K
https://www.tiktok.com/@nhyyng/video/6862937140355960066 14.5K
https://www.tiktok.com/@nhyyng/video/6862554208386764034 23K
https://www.tiktok.com/@nhyyng/video/6861621293440060673 13K
https://www.tiktok.com/@nhyyng/video/6861413901955828993 15.5K
https://www.tiktok.com/@nhyyng/video/6861198903853845761 10.4K
https://www.tiktok.com/@nhyyng/video/6858851467193748737 13.7K
https://www.tiktok.com/@nhyyng/video/6858441012050676994 10.8K
https://www.tiktok.com/@nhyyng/video/6858216103919783169 12K
https://www.tiktok.com/@nhyyng/video/6857311242940452097 9651
https://www.tiktok.com/@nhyyng/video/6848482123821665538 19.2K
https://www.tiktok.com/@nhyyng/video/6846704371267276034 23.

In [134]:
driver = webdriver.Chrome() #chrome_options=chrome_options)
driver.get("https://www.facebook.com/login/")

In [135]:

base_url = 'https://www.facebook.com/100006045494111/videos/781616739195419'
url = ('view-source:' + base_url)
driver.get(url)

data_j = driver.find_element_by_tag_name('body').text

In [136]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(data_j)

In [137]:
soup.prettify()

'<html>\n <body>\n  <p>\n   Line wrap\n   <!DOCTYPE html>\n  </p>\n  <link data-badged-icon="https://static.xx.fbcdn.net/rsrc.php/ye/r/Ta8_J_nYekI.ico" data-default-icon="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" href="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" rel="shortcut icon"/>\n  <meta content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no" name="viewport"/>\n  <link crossorigin="use-credentials" href="/data/manifest/" rel="manifest"/>\n  <style nonce="8QNpzrMt">\n   :root,.__fb-light-mode{--fds-black:#000000;--fds-black-alpha-05:rgba(0, 0, 0, 0.05);--fds-black-alpha-10:rgba(0, 0, 0, 0.1);--fds-black-alpha-15:rgba(0, 0, 0, 0.15);--fds-black-alpha-20:rgba(0, 0, 0, 0.2);--fds-black-alpha-30:rgba(0, 0, 0, 0.3);--fds-black-alpha-40:rgba(0, 0, 0, 0.4);--fds-black-alpha-50:rgba(0, 0, 0, 0.5);--fds-black-alpha-60:rgba(0, 0, 0, 0.6);--fds-black-alpha-80:rgba(0, 0, 0, 0.8);--fds-blue-05:#ECF3FF;--fds-blue-30:#AAC9FF;--fds-blue-40:#77A

In [128]:
info = soup.find("script", type="application/json").contents[0]

In [139]:
for i in soup.find_all("script"):
    print(i)

<script nonce="8QNpzrMt">__DEV__=0;function envFlush(a){function b(b){for(var c in a)b[c]=a[c]}window.requireLazy?window.requireLazy(["Env"],b):(window.Env=window.Env||{},b(window.Env))}envFlush({"routing_namespace":"fb_comet","gk_nonjs_deps_in_require":true,"gk_requirelazy_mem":true,"gk_no_ref_action":true,"compat_iframe_token":"AQ7c6B_2Oc6WXKio0VE"});__annotator=function(f){return f};__d_stub=[];__d=function(){__d_stub.push(arguments)};__rl_stub=[];requireLazy=function(){__rl_stub.push(arguments)};now_inl=(function(){var p=window.performance;return p&&p.now&&p.timing&&p.timing.navigationStart?function(){return p.now()+p.timing.navigationStart}:function(){return new Date().getTime()};})();_btldr={};qpl_inl=(function(){var data={};return function(i,n,t){if(!i)return data;if(!data[i])data[i]={};data[i][n]=t||now_inl();}})();qpl_tag=(function(){var data=[];return function(t){if(!t)return data;data.push(t);}})();qpl_inl("6997994058916842145","htmlStart");qpl_inl("6997994058916842145-serve

In [129]:
json_data = json.loads(info) #["stats"]["playCount"])

In [130]:
for key, value in json_data.items():
    print(key)

props
page
query
buildId
assetPrefix
isFallback
customServer
gip
appGip


In [131]:
for key, value in json_data["props"]["pageProps"]["itemInfo"]["itemStruct"]["video"].items():
    print(key)

id
height
width
duration
ratio
cover
originCover
dynamicCover
playAddr
downloadAddr
shareCover
reflowCover
bitrate
encodedType
format
videoQuality
encodeUserTag
codecType
definition


In [132]:
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))
            
pretty(json_data)

props
	initialProps
		statusCode
			0
		$wid
			6997963359707547138
		$encryptedWebid
			1%7Cjl2tJCfp3H6MLpdy5AblJF-1sA88MC75bvbFEXEihBM%7C1629340395%7C8172a5e77723f2142ee69b8bcf6b05420b327e5a6faec2949a3ad9f089a93936
		$host
			www.tiktok.com
		$pageUrl
			/@ducvamy/video/6995087151092321563?lang=vi-VN&is_copy_url=0&is_from_webapp=v1&sender_device=pc&sender_web_id=6983582057944237570
		$fullUrl
			https://www.tiktok.com/@ducvamy/video/6995087151092321563?lang=vi-VN&is_copy_url=0&is_from_webapp=v1&sender_device=pc&sender_web_id=6983582057944237570
		$isMobile
			False
		$isAndroid
			False
		$isIOS
			False
		$needFix
			False
		$requestId
			16293404033359447981
		$cookieConsent
			ga
				True
			af
				True
			fbp
				True
			lip
				True
		$deviceLimitRegisterExpired
			True
		$slardarAlpha
			enable
				True
			pageId
				['others_homepage']
			gcp
				https://sf16-scmcdn-gcp.ibytedtos.com/goofy/slardar/fe/sdk_lite/browser-legacy.lite.0.0.1-alpha.7.india.js
			va
				https://sf16-sc

			
		itemInfo
			itemStruct
				id
					6995087151092321563
				desc
					Chắc vẫn muốn đòi đi mua thêm váy 🙂 #yody #fyp #couple
				createTime
					1628670646
				scheduleTime
					0
				video
					id
						6995087151092321563
					height
						1024
					width
						576
					duration
						13
					ratio
						720p
					cover
						https://p16-sign-va.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/f6e4d86e1c534af69c89d11467550137?x-expires=1629360000&x-signature=VJs%2Fz5QXoz4HpEzv1kTlYCuUees%3D
					originCover
						https://p16-sign-va.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/c7fe54fd8d5e490d9637cdb5e83b0dc4_1628670647?x-expires=1629360000&x-signature=X2U18cAT%2BJNHRr%2FIcrYJlXHHwW4%3D
					dynamicCover
						https://p16-sign-va.tiktokcdn.com/obj/tos-useast2a-p-0037-aiso/ff2ece7ae16248afa4a6161c328f93fa_1628670648?x-expires=1629360000&x-signature=mbWoj9X%2FY8fgLD3W4pO3dPF2tA8%3D
					playAddr
						https://v16-web.tiktok.com/video/tos/useast2a/tos-useast2a-pve-0037-aiso/e287bd82a98441e

In [133]:
json_data["props"]["pageProps"]["itemInfo"]["itemStruct"]["stats"]["playCount"]


209600

In [ ]:
playCount stats video itemStruct itemInfo pageProps